In [ ]:
!pip install pyspark

In [ ]:
!wget https://jpbarddal.github.io/assets/data/bigdata/transactions_amostra.csv.zip
!unzip transactions_amostra.csv.zip

In [ ]:
from pyspark.sql import SparkSession

# Inicializa o Spark
spark = SparkSession.builder\
    .master('local[*]')\
    .appName("")\
    .getOrCreate()
    
sc = spark.sparkContext

# Carrega o conjunto de dados a partir do arquivo CSV
dataset = sc.textFile("transactions_amostra.csv")

In [ ]:
# Printando as colunas

columns = dataset.take(1)[0].split(";")
print("Colunas do dataset:")
for column in columns:
    print(column)


#Problema 1: O número de transações envolvendo o Brasil

> Bloco com recuo




In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tde2").getOrCreate()
sc = spark.sparkContext

df = sc.textFile('transactions_amostra.csv')

lines = df.map(lambda line: line.split(";"))

brazil = lines.filter(lambda line: line[0] == 'Brazil')

count = brazil.count()

print(count)

#Problema 2: O número de transações por tipo de fluxo e ano


In [ ]:
# Mapeia as transações para o par ((tipo de fluxo, ano), 1)
transacoes_por_fluxo_ano = dataset.map(lambda linha: ((linha.split(";")[4], linha.split(";")[1]), 1)) \
    .reduceByKey(lambda a, b: a + b)

# Itera sobre os resultados e imprime o tipo de fluxo, ano e contagem de transações
for fluxo_ano, contagem in transacoes_por_fluxo_ano.collect():
    print("Tipo de fluxo e ano:", fluxo_ano, contagem)

# Problema 3: A média dos valores das commodities por ano



In [ ]:
# Filtra as linhas que possuem um valor numérico válido na coluna de valores de commodities
valores_commodities = dataset.filter(lambda linha: linha.split(";")[5].replace(".", "").isdigit()) \
    .map(lambda linha: (linha.split(";")[1], float(linha.split(";")[5])))

# Calcula a soma e contagem dos valores das commodities por ano usando a função aggregateByKey
valores_commodities_por_ano = valores_commodities.aggregateByKey((0.0, 0),
                                                                  lambda acumulador, valor: (acumulador[0] + valor, acumulador[1] + 1),
                                                                  lambda acumulador1, acumulador2: (acumulador1[0] + acumulador2[0], acumulador1[1] + acumulador2[1]))

# Calcula a média dos valores das commodities por ano usando a função mapValues
media_valores_por_ano = valores_commodities_por_ano.mapValues(lambda acumulador: acumulador[0] / acumulador[1])

# Itera sobre os resultados e imprime o valor médio da commodity para cada ano
for ano, media in media_valores_por_ano.collect():
    print("Valor médio da commodity para o ano", ano, ":", media)

# Problema 4: O preço médio das commodities por tipo de unidade, ano e categoria no fluxo de exportação no Brasil


In [ ]:
df = sc.textFile('transactions_amostra.csv')

pairrdd = df.filter(lambda x: "Brazil" in x and x.split(';')[4] == "Export")
pairrdd = pairrdd.map(lambda x: ((x.split(';')[1], x.split(';')[7], x.split(';')[3], x.split(';')[8]), float(x.split(';')[5])))

soma = pairrdd.aggregateByKey((0.0, 0), lambda x,y: (x[0] + y, x[1] + 1), lambda x,y: (x[0] + y[0], x[1] + y[1]))
media = soma.mapValues(lambda x: x[0]/x[1])

media.take(5)

# Problema 5: O preço máximo, mínimo e médio por tipo de unidade e ano


In [ ]:
# Processar o conjunto de dados
transacoes = dataset.map(lambda linha: ((linha.split(";")[7], linha.split(";")[1]), float(linha.split(";")[5].replace(",", ".")) if linha.split(";")[5].replace(",", ".").replace(".", "").isdigit() else 0))

# Filtrar transações com preços inválidos
transacoes_validas = transacoes.filter(lambda transacao: transacao[1] != 0)

# Calcular preço máximo, mínimo e médio por tipo de unidade e ano
estatisticas_preco_por_unidade_ano = transacoes_validas.aggregateByKey(
    (float('-inf'), float('inf'), 0.0, 0),
    lambda a, b: (max(a[0], b), min(a[1], b), a[2] + b, a[3] + 1),
    lambda a, b: (max(a[0], b[0]), min(a[1], b[1]), a[2] + b[2], a[3] + b[3])
)

# Calcular preço médio da transação
preco_medio_por_unidade_ano = estatisticas_preco_por_unidade_ano.mapValues(lambda v: (v[0], v[1], v[2] / v[3]))

# Iterar sobre os resultados e imprimir preço máximo, mínimo e médio por tipo de unidade e ano
for (unidade_ano, (preco_maximo, preco_minimo, preco_medio)) in preco_medio_por_unidade_ano.collect():
    tipo_unidade, ano = unidade_ano
    print("Tipo de Unidade:", tipo_unidade)
    print("Ano:", ano)
    print("Preço Máximo:", preco_maximo)
    print("Preço Mínimo:", preco_minimo)
    print("Preço Médio:", preco_medio)
    print()

# Problema 6: O país com o maior preço médio de commodities na categoria Exportação


In [ ]:
# Filtra o conjunto de dados para a categoria de exportação
exportacao = dataset.filter(lambda line: line.split(";")[4] == "Export")

# Mapeia cada linha para pares chave-valor (país, preço)
preco_por_pais = exportacao.map(lambda line: (line.split(";")[0], float(line.split(";")[5])))

# Calcula a soma e contagem dos preços para cada país
soma_contagem_por_pais = preco_por_pais.aggregateByKey(
    (0.0, 0),
    lambda acc, price: (acc[0] + price, acc[1] + 1),
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
)

# Calcula o preço médio para cada país
preco_medio_por_pais = soma_contagem_por_pais.mapValues(lambda acc: acc[0] / acc[1])

# Encontra o país com o maior preço médio
pais_com_maior_preco_medio = preco_medio_por_pais.max(lambda x: x[1])

# Imprime o resultado
print("País com o maior preço médio de commodities na categoria Exportação:",
      pais_com_maior_preco_medio[0], pais_com_maior_preco_medio[1])

# Problema 7: A commodity mais comercializada (somando as quantidades) em 2016 por tipo de fluxo


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tde2").getOrCreate()
sc = spark.sparkContext

df = sc.textFile('transactions_amostra.csv')

pairrdd = df.filter(lambda x: x.split(';')[1] == "2016")

pairrdd = pairrdd.map(lambda x: (x.split(';')[3], (x.split(';')[4], float(x.split(';')[8]))))

export = pairrdd.filter(lambda x: x[1][0] == "Export")
export = export.map(lambda x: (x[0], x[1][1]))

somaExport = export.reduceByKey(lambda x,y: x+y)
maiorExport = somaExport.max(lambda x: x[1])

importa = pairrdd.filter(lambda x: x[1][0] == "Import")
importa = importa.map(lambda x: (x[0], x[1][1]))

somaImporta = importa.reduceByKey(lambda x,y: x+y)
maiorImporta = somaImporta.max(lambda x: x[1])

ReExport = pairrdd.filter(lambda x: x[1][0] == "Re-Export")
ReExport = ReExport.map(lambda x: (x[0], x[1][1]))

somaReExport = ReExport.reduceByKey(lambda x,y: x+y)
maiorReExport = somaReExport.max(lambda x: x[1])

ReImport = pairrdd.filter(lambda x: x[1][0] == "Re-Import")
ReImport = ReImport.map(lambda x: (x[0], x[1][1]))

somaReImport = ReImport.reduceByKey(lambda x,y: x+y)
maiorReImport = somaReImport.max(lambda x: x[1])

print("Export")
print(maiorExport)
print("Import")
print(maiorImporta)
print("Re-Export")
print(maiorReExport)
print("Re-Import")
print(maiorReImport)